In [1]:
from gensim.models import word2vec
import pandas as pd

In [2]:
#data preparation
data_df = pd.read_csv('order_products__train.csv')

data_dict = {}

for index,row in data_df.iterrows():
    if row['order_id'] not in data_dict.keys():
        data_dict[row['order_id']] = [str(row['product_id'])]
    else:
        data_dict[row['order_id']].append(str(row['product_id']))

    

In [3]:
data_list = [v for _,v in data_dict.items()]


In [4]:
model = word2vec.Word2Vec(data_list, size = 300, min_count = 1, workers = 10)

model.train(data_list, total_examples = len(data_list), epochs = 20)

(26668167, 27692340)

In [5]:
model.wv.most_similar(positive='19204') 
# input:
# 19204 - organic flakes

# results:
# 4410 - organic flakes
# 18224 - organic yogurt
# 25767 - organic cereal
# 340 - eco cereal
# 48007 - cereal
# 49241 - organic oatmeal
# 36914 - organic juice
# 49111 - oatmeal
# 32648 - gluten free granola
# 10831 - organic yogurt

/home/pfalkiewicz/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('18224', 0.6081624031066895),
 ('4410', 0.6055404543876648),
 ('45544', 0.5636848211288452),
 ('340', 0.5567233562469482),
 ('36914', 0.5518804788589478),
 ('34019', 0.551815390586853),
 ('20460', 0.5484035611152649),
 ('48007', 0.5467056035995483),
 ('49111', 0.5430724620819092),
 ('25767', 0.5423418879508972)]

In [6]:
import math
import numpy as np
def power_means(list_of_vectors):
    temp1,temp2,temp3 = [],[],[]
    for it in range(len(list_of_vectors[0])):
        tempC1 = 0
        tempC2 = []
        tempC3 = 0
        for vector in list_of_vectors:
            tempC1 += vector[it]
            tempC2.append(vector[it])
            tempC3 += 1/vector[it]
        temp1.append(tempC1/len(vector))
        temp2.append((np.sign(tempC2)*(np.abs(np.prod(tempC2)))**(1/len(tempC2)))[0])
        temp3.append(len(vector)/tempC3)
    
    return temp1 + temp2 + temp3
            

In [7]:
data_dict_pm = {}

for transaction_key in data_dict:
    data_dict_pm[transaction_key] = power_means([model[product] for product in data_dict[transaction_key]])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [8]:
# save vectors
col_key = [k for k,v in data_dict.items()]
col_value = [data_dict_pm[key] for key in col_key]
transactions_df = pd.DataFrame({'order_id':col_key, 'order_vector': col_value})
transactions_df.to_csv('transactions_vectors.csv', index = None)

In [9]:
#generate sample queries
import random
length_of_queries = 100
sample_query_uni = [random.choice(random.choice([v for _,v in data_dict.items()])) for i in range(length_of_queries)]

# A - products from one query
# B - products from different queries

sample_query_bigramA = []
for i in range(length_of_queries):
    temp = []
    transaction = random.choice([v for v in data_list if len(v) > 1])
    temp.append(random.choice(transaction))
    temp.append(random.choice(transaction))
    sample_query_bigramA.append(temp)
    
sample_query_bigramB = []
for i in range(length_of_queries):
    temp = []
    transaction1 = random.choice([v for v in data_list if len(v) > 1])
    transaction2 = random.choice([v for v in data_list if len(v) > 1])
    temp.append(random.choice(transaction1))
    temp.append(random.choice(transaction2))
    sample_query_bigramB.append(temp)
        
sample_query_trigramA = []
for i in range(length_of_queries):
    temp = []
    transaction = random.choice([v for v in data_list if len(v) > 2])
    temp.append(random.choice(transaction))
    temp.append(random.choice(transaction))
    temp.append(random.choice(transaction))
    sample_query_trigramA.append(temp)
    
sample_query_trigramB = []
for i in range(length_of_queries):
    temp = []
    transaction1 = random.choice([v for v in data_list if len(v) > 2])
    transaction2 = random.choice([v for v in data_list if len(v) > 2])
    transaction3 = random.choice([v for v in data_list if len(v) > 2])
    temp.append(random.choice(transaction1))
    temp.append(random.choice(transaction2))
    temp.append(random.choice(transaction3))
    sample_query_trigramB.append(temp)

In [10]:
queries = {}

for query in sample_query_uni:
    queries[query] = power_means([model[query]])
    
for query in sample_query_bigramA:
    queries[str(query)] = power_means([model[element] for element in query])

for query in sample_query_bigramB:
    queries[str(query)] = power_means([model[element] for element in query])    
    
for query in sample_query_trigramA:
    queries[str(query)] = power_means([model[element] for element in query])    
    
for query in sample_query_trigramA:
    queries[str(query)] = power_means([model[element] for element in query])    

    
for k,v in queries.items():
    print(k)
    print(v)
    break

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method

['2110', '16953']
[0.0006832771748304367, -0.0002395890404780706, 0.006328729838132858, 0.004111267626285553, 0.003698736064446469, -0.004203174064556758, -0.001362531433502833, 0.0003454497860123714, 0.003027005543311437, -0.0019434981544812521, -0.0022985349595546723, 0.0015325477719306945, -0.002673344115416209, -0.0034997756282488506, 0.0014457933977246285, 0.005034311873217423, 0.0021678522974252702, 0.0002149977907538414, -0.0024006522198518118, -0.0032913629213968914, -0.005215474367141724, 0.0026839133103688556, -0.004041018585364024, 0.00018258363008499145, 0.0014068898061911266, -0.00038045340528090795, 0.002006514718135198, -0.002301695682108402, -0.005339941531419754, 0.0020036450525124866, -0.001985027380287647, -0.0019410030543804168, 0.0021346686120765905, -0.0010270173847675324, -0.0034007962544759115, -0.002231874105830987, 0.0023560032372673353, 0.0029220006863276164, -0.003938898717363675, -0.002119687745968501, -0.004090459967652956, -0.0019199193517367046, 0.003665

In [15]:
products_df = pd.read_csv('products.csv')

prod2id  = {}
for _, row in  products_df.loc[:,['product_id','aisle_id']].iterrows():
    prod2id[str(row['product_id'])] = row['aisle_id']


In [16]:
item = '19204'
list_of = model.wv.most_similar(positive=item) 

/home/pfalkiewicz/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [20]:
print(prod2id[item])

for row in list_of:
    print(prod2id[row[0]])

121
120
121
120
121
98
130
130
121
130
121


0.9999999999999998